<a href="https://colab.research.google.com/github/stella-depuydt/j23_cs167_notes/blob/main/Day03_P1_kNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day03 
## Part 1: k-Nearest-Neighbor

#### CS167: Machine Learning, J-Term 2023

Thursday, January 5th, 2023 -- Session I (9:00-10:15)

[⏮ Day02 Part 2](https://github.com/merriekay/j23_cs167_notes/blob/main/Day02_P2_Pandas_Exercises.ipynb) | [Day03 Part 2⏩]()

## Helpful Links:
📆 [Course Schedule](https://docs.google.com/spreadsheets/d/e/2PACX-1vStj3FCEJqloUMLn2VtHa4yy1ILY6WvABhu4jd4cVUpPGkrx1mEjfTFmd77DMESR9HJ-8UBxgMDJL06/pubhtml?gid=0&single=true) | 🙋[PollEverywhere](https://pollev.com/meredithmoore011) | 📜 [Syllabus](https://analytics.drake.edu/~moore/j23_cs167/Syllabus.html)


# Overview of Today:

Part 1: Notebook #1 questions, k-Nearest Neighbros

Part 2: Missing Data Handling, Normalization

# Admin Stuff

You should be working on:
- Notebook #1, is due tonight by 11:59 pm. 
    - to submit, copy and paste the link to your github repository to Blackboard. My link would look like: https://github.com/j23-cs167/notebook-1-python-pandas-and-pokemon-merriekay, but yours will have your GitHub username rather than 'merriekay'
- [Notebook #2: kNN and Normalization](https://classroom.github.com/a/ZihGOnY-) is released today, but will be due on Friday 1/6 by 11:59pm.
- Heads up that Quiz #1 will be released on Friday after class and will be due Monday by 11:59pm. 


## Before we get started, let's load in our datasets:

In [ ]:
#run this cell if you're using Colab:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import the data:
#make sure the path on the line below corresponds to the path where you put your dataset.
import pandas as pd
path = 'datasets/titanic.csv'
titanic = pd.read_csv(path)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
path = 'datasets/irisData.csv'
iris= pd.read_csv(path)
iris.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 🚩 Notebook #1 Help

A few helpful functions for Notebook #1:
- `max()` will return the maximum value in a Series
- `idxmax()` will return the __index__ of the maximum value 

## Other functions worth mentioning:
- `min()` will return the minimum value in a Series
- `mode()` will return the mode--the most common value--of the dataset

# 💬 Review Exercise:

Using the Titanic dataset, find how much the oldest passenger on the Titanic paid. 

## Quick overview:
So far, we've talked about:
- Intro to ML
- Python review
- Introduction to Pandas
- Handling Missing Data

## Quick Simple Stats Review:

- __mean__: the average, take the sum of elements, and divide by the number of total elements
- __median__: The middle number; found by ordering all data poits and picking out the one in the middle (or if there are two numbers in the middle, taking the mean of those two numbers). 
- __mode__: The most frequent number; the number that occurs the highestnumber of times. 

### [2,2,3,3,4,4,4]
Find the:
- mean:
- median:
- mode:

## Machine Learning Variations

We are going to learn about a lot of different types of machine learning in CS167. Here are a few categories to look out for: 
- __classifcation__: identify which category it goes in. Examples: Spam or ham? Eric or Tim? Fish, amphibian, reptile, bird, or mammal
- __regression__: real-valued labels. Examples: price of Bitcoin, tomorrow's temperature, etc.
- __supervised learning__: data has labels, goal is to predict the labels of new instance. 
- __unsupervised learning__: data does not have a label, the goal is to analyze/cluster the examples. 
- __other issues__: missing data, sequential data, outlier anomaly detetion, and many more. 

## Let's get our data loaded and ready:

## PollEverywhere Q:

Let's take a look at a couple rows of the Iris Dataset:

In [ ]:
iris.head(2)

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


## 🚨 Terminology Alert 🚨
Each row in the table represents a __training example__, a previously-seen, known instance of the thing we are trying to model. 

Each column in the table represents a __feature__, some attribute or variable that each training example has a value for. 

__Target variable__: the 'feature' we will try to predict(e.g. species)--it's value is unkonwn for any new cases not in the training data.

__Predictor variables__: (or just predictors), the features that will be used to make predictions of the target variable. (e.g. `sepal length`, `petal length`, `sepal width`, `petal width`

Remember this question from Day01?

- 5.1 cm petal, 7.2 cm sepal

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day0_iris_q1.png?raw=1"/>
</div>


# ✨New Material: 

# Our First Machine Learning Model: k Nearest Neighbors
We're starting with a relatively simple, but foundational, machine learning model, the k-Nearest Neighbors (kNN) algorithm. 

__k-Nearest-Neighbor Algorihm__: Precict the _most commonly appearing_ class among the __k__ closest training examples.

## 1-Nearest-Neighbor Algorithm

__k-Nearest-Neighbor Algorihm__: Precict the _most commonly appearing_ class among the __k__ closest training examples.
- in a 1-NN algorithm k=1

### What will a 1NN algorithm predict?


<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_1NN_iris.png?raw=1" width=450/>
</div>

It will predict __Iris-versicolor__ because the nearest neighbor (shown by the black line) is Iris-versicolor. 

## 3-Nearest-Neighbor Algorithm

> Wait... why did we skip 2-NN?

### What will a 3NN algorithm predict?

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_3NN_iris.png?raw=1" width=450/>
</div>

In a 3-Nearest Neighbor algorithm k=3. So, let's look at the 3 closest neighbor to our iris.
- The 3 cloesest neighbors are: `Iris-versicolor, Iris-virginica, Iris-virginica`
- a 3NN algorithm will predict the _most commonly appearing class_ among the k closest neighbors, so it will predict __Iris-virginica__.

## What do you mean by "closest"?
__k-Nearest-Neighbor Algorihm__: Precict the _most commonly appearing_ class among the __k__ closest training examples.

Alright... but how do we determine which training examples are the 'closest'?

__defining 'nearness':__ as the machine learning engineer, we get to choose how we define _close_. 
- What ways can you think of to determine a distane between any two training examples?

# Distances:

## Euclidean Distance:

Example in 2D space: distance between (x1, y1) and (x2, y2) is: 

## $\sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}$

## 2D Eucliden Distance Example:
Find the distance between (1,2) and (4,6):

$\sqrt{(1 - 4)^2 + (2 - 6)^2}$

$\sqrt{(-3)^2 + (-4)^2}$

$\sqrt{9 + 16}$

$\sqrt{25}$

$5$


## Euclidean Distance in n-Dimensional Space:

Suppose features of an example $x$ are denoted $\langle a_1(x), a_2(x), ... , a_n(x) \rangle$
- where $n$ is the number of features.

Then the Euclidean Distance:

## $d(x_i, x_j) = \sqrt{ \sum_{r=1}^{n} (a_r(x_i) - a_r(x_j))^2}$

## $d(x_i, x_j) = \sqrt{ \sum_{r=1}^{n} (a_r(x_i) - a_r(x_j))^2}$

Example: Suppose we find a new iris with:
- 7.2 cm sepal length, 2.5 cm sepal width,
- 5.1 cm petal length and 1.5 cm petal width.

Here are some rows from the training data:

| sepal length | sepal width | petal length | petal width | species        |
|--------------|-------------|--------------|-------------|----------------|
| 4.6          | 3.2         | 1.4          | 0.2         | Iris-setosa    |
| 6.2          | 2.8         | 4.8          | 1.8         | Iris-virginica |

## Euclidean Distance Example:
<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_euclidean_ex.png?raw=1"/>
</div>

## Group Exercise:

Unknown Instance:
- 7.2 cm sepal length, 2.6 cm sepal width
- 5.1 cm petal length and 1.5 cm petal width.

If this was our entire set of training examples:

| sepal length | sepal width | petal length | petal width | species         |
|--------------|-------------|--------------|-------------|-----------------|
| 4.6          | 3.2         | 1.4          | 0.2         | Iris-setosa     |
| 6.2          | 2.8         | 4.8          | 1.8         | Iris-virginica  |
| 6.2          | 2.2         | 4.5          | 1.5         | Iris-versicolor |
| 6.3          | 2.7         | 4.9          | 1.8         | Iris-virginica  |

### What would the 1-NN version predict? 2-NN? 3-NN? 4-NN?

## Distance Functions
You can use any distance function you want. Other common distances include:

### Manhattan Distance: 
### $d(x_i, x_j) = \sum_{r=1}^{n} | a_r(x_i) - a_r(x_j)|$

### Minkowski distance: 
### $d(x_i, x_j) = (\sum_{r=1}^{n} | a_r(x_i) - a_r(x_j)|^p)^{1/p}$

### Or you can make your own:
- Online dating use distance scores to predict who you will swipe left/right on
- Baseball similarity matrix
- etc.

# 💬 Discussion Question:

What do we do if the features aren't numbers?
- like Titanic `embark_town`... how can we calculate a distance between `Southampton` and `Queenstown`?

In [ ]:
path = 'datasets/titanic.csv'
titanic = pd.read_csv(path)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
pd.get_dummies(titanic.embark_town)

,Cherbourg,Queenstown,Southampton
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
886,0,0,1
887,0,0,1
888,0,0,1
889,1,0,0


# 💬 Discussion Question:

What if our __target variable__ is continuous rather than categorical? How would we make a prediction using kNN? 
- Can we do regression with kNN? If so, how?

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_btc_price.jpeg?raw=1" width = 500/>
</div>

# Let's implement kNN in Python/Pandas: 

Let's build a 5-nearest-neighbor Iris classifier:

To implement this 5NN, we need to do 3 things:
1. Calculate the distances from each of the rows to the new instance
2. Sort the data by these distances 
3. Select the k closest training examples and use them to predict the most commonly occuring class of the closest neighbors.

## 1. Calculate the Distances:
Let's start by adding a new column to our `iris` dataframe that is the distance from each existing row to the new instance with:
- 5.1 petal length, 7.2 sepal length, 1.5 petal width, and 2.5 sepal width

In [ ]:
import numpy as np

iris['distance_to_new'] = np.sqrt(
    (5.1 - iris['petal length'])**2 
    +(7.2 - iris['sepal length'])**2 
    +(1.5 - iris['petal width'])**2
    +(2.5 - iris['sepal width'])**2)

iris.head()

,sepal length,sepal width,petal length,petal width,species,distance_to_new
0,5.1,3.5,1.4,0.2,Iris-setosa,4.559605
1,4.9,3.0,1.4,0.2,Iris-setosa,4.573839
2,4.7,3.2,1.3,0.2,Iris-setosa,4.782259
3,4.6,3.1,1.5,0.2,Iris-setosa,4.665833
4,5.0,3.6,1.4,0.2,Iris-setosa,4.629255


## 2. Sort the data by the distance
Let's now sort our data using the built in `sort_values()` function. 

We want to find the nearest k neighbors, so sorting them in ascending order (which is the default setting for `sort_values()` will work nicely. 

In [ ]:
sorted_data = iris.sort_values(['distance_to_new'])
sorted_data.iloc[0:5]

,sepal length,sepal width,petal length,petal width,species,distance_to_new
76,6.8,2.8,4.8,1.4,Iris-versicolor,0.591608
52,6.9,3.1,4.9,1.5,Iris-versicolor,0.700000
77,6.7,3.0,5.0,1.7,Iris-versicolor,0.741620
50,7.0,3.2,4.7,1.4,Iris-versicolor,0.836660
129,7.2,3.0,5.8,1.6,Iris-virginica,0.866025


## 3. Display the most common species among these 5


In [ ]:
sorted_data.iloc[0:5]['species'].mode()

0    Iris-versicolor
dtype: object

And Viola! We have successfully implemented our first machine learning model from scratch.

## Questions to consider:

What part of the code would you change to use some other value of k in the kNN?

What would you change if you wanted to classify a different new Iris?

## Quick Function Refresher:

In [ ]:
# reminder of what a function looks like in Python: 

def find_min(a_list):
    smallest = float("inf")
    for element in a_list:
        if element < smallest:
            smallest=element
    return smallest

In [ ]:
find_min([2,5, -4, 54,14, 30])

-4

# Programming Exercise:

Rewrite our k-NN code so tht it's a function.

Pass the iris measurements (specimen), dataframe, and k as parameters and return the predicted class.

Here's some starter code:

In [ ]:
def kNN(specimen, data, k):
    # write your code in here to make this function work
    
    
    prediction = 'iris'
    return prediction

# Admin Stuff

You should be working on:
- Notebook #1, is due tonight by 11:59 pm. 
    - to submit, copy and paste the link to your github repository to Blackboard. My link would look like: https://github.com/j23-cs167/notebook-1-python-pandas-and-pokemon-merriekay, but yours will have your GitHub username rather than 'merriekay'
- [Notebook #2: kNN and Normalization](https://classroom.github.com/a/ZihGOnY-) is released today, but will be due on Friday 1/6 by 11:59pm.
- Heads up that Quiz #1 will be released on Friday after class and will be due Monday by 11:59pm. 
